### Data Description
The skill builder dataset has 30 columns.
Please refer to this [link](https://sites.google.com/site/assistmentsdata/home/assistment-2009-2010-data) for detail description.

**The relevant columns are:**
- order_id: it is chronological.
- user_id: the id of the student doing the problem.
- problem_id: the id of the problem
- correct: 1 means correct on the first attempt, 0 means incorrect on the first attempt, or asked for help

**The following columns are useful but may not be used for DKT:**
- skill_id: the skill associated with the problem. 
- **orginal: 1 means main problem, 0 means scaffolding problem**
    - It is required to determine whether to include scaffolding
- ms_first_response: The time in milliseconds for the student's first response.
- hint_count: number of student attempts on this problem.
- attempt_count: number of student attmepts on this problem.

---
The following code will use numpy and pandas to process the **2009-2010 ASSISTment Data** so as to convert it into a tensorflow-friendly data file.

In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import logging
import csv

LOGGER = logging.getLogger(__name__)
file_path = './data/skill_builder_data.csv'

scaffolding = False
empty_skill = True

# encoding are required as it is not utf8 encoded.
data = pd.DataFrame.from_csv(file_path, encoding='ISO-8859-1')

C:\Users\YEUNG\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2825: DtypeWarning: Columns (17,29) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [25]:
num_users = len(data.user_id.unique())
num_problems = len(data.problem_id.unique())
num_records = data.shape[0]
msg = "In this dataset, there are {0} records, with {1} students and {2} \
different questions."
print(msg.format(num_records, num_users, num_problems))
print("With the following columns: \n", data.columns)

In this dataset, there are 525534 records, with 4217 students and 26688 different questions.
With the following columns: 
 Index(['assignment_id', 'user_id', 'assistment_id', 'problem_id', 'original',
       'correct', 'attempt_count', 'ms_first_response', 'tutor_mode',
       'answer_type', 'sequence_id', 'student_class_id', 'position', 'type',
       'base_sequence_id', 'skill_id', 'skill_name', 'teacher_id', 'school_id',
       'hint_count', 'hint_total', 'overlap_time', 'template_id', 'answer_id',
       'answer_text', 'first_action', 'bottom_hint', 'opportunity',
       'opportunity_original'],
      dtype='object')


### Processing the data
1. Filter out students with exactly one interaction.

In [3]:
def generate_id_to_idx_dict(df, column):
    ids = df[column].unique()
    num_unique_ids = len(ids)
    id_to_idx_dict = dict(zip(ids, range(num_unique_ids)))
    return id_to_idx_dict

In [40]:
REQUIRE_COLS = ['time_idx', 'user_id', 'skill_id', 'correct']

# get the time index
data['time_idx'] = data.index.values
data.head()

# remove nan in skill_id
if not empty_skill:
    # remove nan in skill_id
    nan_records = data.skill_id.apply(np.isnan)
    data = data[~nan_records]
    print("The data shape after remove nan:", data.shape)
else:
    # replace nan with 0 in skill_id
    data['skill_id'] = data['skill_id'].fillna(0)

if not scaffolding:
    data = data[data.original == 1]
    print("The data shape after remove scaffording:", data.shape)
    

# remove duplicated records
columns = set(data.columns.values)
columns.remove('opportunity')
columns.remove('opportunity_original')
columns = list(columns)
data = data[~data.duplicated(subset=columns)]
print("The data shape after remove duplicated records:", data.shape)

The data shape after remove scaffording: (449220, 30)
The data shape after remove duplicated records: (328291, 30)


In [41]:
data.skill_id.unique()

array([   1.,    2.,    4.,    5.,    8.,    9.,   10.,   11.,   12.,
         13.,   14.,   15.,   16.,   17.,   18.,   21.,   22.,   24.,
         25.,   26.,   27.,   32.,   34.,   35.,   37.,   39.,   40.,
         42.,   43.,   46.,   47.,   48.,   49.,   50.,   51.,   53.,
         54.,   58.,   61.,   63.,   64.,   65.,   67.,   69.,   70.,
         74.,   75.,   76.,   77.,   79.,   80.,   81.,   82.,   83.,
         84.,   85.,   86.,   91.,   92.,   94.,   96.,   97.,   99.,
        101.,  102.,  104.,  105.,  110.,  163.,  165.,  166.,  173.,
        190.,  193.,  203.,  204.,  217.,  221.,  276.,  277.,  278.,
        279.,  280.,  290.,  292.,  293.,  294.,  295.,  296.,  297.,
        298.,  299.,  301.,  303.,  307.,  308.,  309.,  310.,  311.,
        312.,  314.,  317.,  321.,  322.,  323.,  324.,  325.,  331.,
        333.,  334.,  340.,  343.,  346.,  348.,  350.,  356.,  362.,
        365.,  367.,  368.,  371.,  375.,  378.,    0.])

In [45]:
skill_df = data[['skill_id', 'skill_name']].drop_duplicates()
idx_to_problem_dict = {}
for index, row in skill_df.iterrows():
    skill_idx = problem_to_idx_dict[row['skill_id']]
    idx_to_problem_dict[skill_idx] = (row['skill_id'], row['skill_name'])

In [44]:
for key, value in idx_to_problem_dict.items():
    print (key, value)

0 (1.0, 'Box and Whisker')
1 (2.0, 'Circle Graph')
2 (4.0, 'Histogram as Table or Graph')
3 (5.0, 'Number Line')
4 (8.0, 'Scatter Plot')
5 (9.0, 'Stem and Leaf Plot')
6 (10.0, 'Table')
7 (11.0, 'Venn Diagram')
8 (12.0, 'Mean')
9 (13.0, 'Median')
10 (14.0, 'Mode')
11 (15.0, 'Range')
12 (16.0, 'Counting Methods')
13 (17.0, 'Probability of Two Distinct Events')
14 (18.0, 'Probability of a Single Event')
15 (21.0, 'Interior Angles Figures with More than 3 Sides')
16 (22.0, 'Interior Angles Triangle')
17 (24.0, 'Congruence')
18 (25.0, 'Complementary and Supplementary Angles')
19 (26.0, 'Angles on Parallel Lines Cut by a Transversal')
20 (27.0, 'Pythagorean Theorem')
21 (32.0, 'Nets of 3D Figures')
22 (34.0, 'Unit Conversion Within a System')
23 (35.0, 'Effect of Changing Dimensions of a Shape Prportionally')
24 (37.0, nan)
25 (39.0, 'Area Circle')
26 (40.0, 'Circumference ')
27 (42.0, 'Perimeter of a Polygon')
28 (43.0, 'Reading a Ruler or Scale')
29 (46.0, 'Calculations with Similar Figure

In [18]:
problem_to_idx_dict

{0.0: 123,
 1.0: 0,
 2.0: 1,
 4.0: 2,
 5.0: 3,
 8.0: 4,
 9.0: 5,
 10.0: 6,
 11.0: 7,
 12.0: 8,
 13.0: 9,
 14.0: 10,
 15.0: 11,
 16.0: 12,
 17.0: 13,
 18.0: 14,
 21.0: 15,
 22.0: 16,
 24.0: 17,
 25.0: 18,
 26.0: 19,
 27.0: 20,
 32.0: 21,
 34.0: 22,
 35.0: 23,
 37.0: 24,
 39.0: 25,
 40.0: 26,
 42.0: 27,
 43.0: 28,
 46.0: 29,
 47.0: 30,
 48.0: 31,
 49.0: 32,
 50.0: 33,
 51.0: 34,
 53.0: 35,
 54.0: 36,
 58.0: 37,
 61.0: 38,
 63.0: 39,
 64.0: 40,
 65.0: 41,
 67.0: 42,
 69.0: 43,
 70.0: 44,
 74.0: 45,
 75.0: 46,
 76.0: 47,
 77.0: 48,
 79.0: 49,
 80.0: 50,
 81.0: 51,
 82.0: 52,
 83.0: 53,
 84.0: 54,
 85.0: 55,
 86.0: 56,
 91.0: 57,
 92.0: 58,
 94.0: 59,
 96.0: 60,
 97.0: 61,
 99.0: 62,
 101.0: 63,
 102.0: 64,
 104.0: 65,
 105.0: 66,
 110.0: 67,
 163.0: 68,
 165.0: 69,
 166.0: 70,
 173.0: 71,
 190.0: 72,
 193.0: 73,
 203.0: 74,
 204.0: 75,
 217.0: 76,
 221.0: 77,
 276.0: 78,
 277.0: 79,
 278.0: 80,
 279.0: 81,
 280.0: 82,
 290.0: 83,
 292.0: 84,
 293.0: 85,
 294.0: 86,
 295.0: 87,
 296.0: 88,


In [42]:
user_ids = data.user_id.unique()
problem_to_idx_dict = generate_id_to_idx_dict(data, column='skill_id')

tuples = []
for id in user_ids:
    df = sorted_data[sorted_data.user_id == id]
    df = df[REQUIRE_COLS]
    problems = [problem_to_idx_dict[pid] for pid in df.skill_id]
    corrects = [corr for corr in df.correct]
    num_problems = len(problems)
#     print (num_problems)
#     print (problems)
#     print (corrects)
#     print ("============")
    result = (num_problems, problems, corrects)
    tuples.append(result)

In [6]:
with open('data/b.csv', 'w') as f:
    writer = csv.writer(f, 
                        delimiter=',', 
                        quotechar="'", 
                        quoting=csv.QUOTE_MINIMAL,
                        lineterminator='\n')
    for tup in tuples:
        writer.writerow([tup[0]])
        writer.writerow(tup[1])
        writer.writerow(tup[2])

In [44]:
len(tuples)

4217

In [45]:
len(problem_to_idx_dict.keys())

124

In [26]:
train, test = train_test_split(tuples, test_size=0.2)

In [27]:
with open('data/skill_id_train.csv', 'w') as f:
    writer = csv.writer(f, 
                        delimiter=',', 
                        quotechar="'", 
                        quoting=csv.QUOTE_MINIMAL,
                        lineterminator='\n')
    for tup in train:
        writer.writerow([tup[0]])
        writer.writerow(tup[1])
        writer.writerow(tup[2])

In [28]:
with open('data/skill_id_test.csv', 'w') as f:
    writer = csv.writer(f, 
                        delimiter=',', 
                        quotechar="'", 
                        quoting=csv.QUOTE_MINIMAL,
                        lineterminator='\n')
    for tup in test:
        writer.writerow([tup[0]])
        writer.writerow(tup[1])
        writer.writerow(tup[2])